# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

In [86]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [95]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

In [96]:
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
import re

# 初始化
milvus_client = MilvusClient(uri="./milvus_mfd.db")
collection_name = "mfd_rag_collection"

# 删除旧的 collection
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

# 创建新 collection
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=768,
    metric_type="COSINE",
    consistency_level="Strong"
)

# 加载 & 切分文档
text_lines = []
with open("mfd.md", "r", encoding="utf-8") as f:
    text = f.read()

# 按条文编号切分
chunks = re.split(r'(?=\*\*第[一二三四五六七八九十百]+条\*\*)', text)
for chunk in chunks:
    chunk = chunk.strip()
    if not chunk:
        continue
    lines = chunk.splitlines()
    title = lines[0].strip()
    content = " ".join(line.strip() for line in lines[1:] if line.strip())
    text_lines.append(f"{title} {content}")

print(f"共提取 {len(text_lines)} 个条文")

# 生成向量
embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")
doc_embeddings = embedding_model.encode(text_lines)

# 插入到 Milvus
data = []
for i, line in enumerate(text_lines):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

print("入库完成")


共提取 355 个条文
入库完成


In [97]:
def extract_article_number(question: str):
    """提取问题中的条文编号"""
    m = re.search(r"第二百[一二三四五六七八九十百]+条", question)
    if m:
        return m.group(0)
    return None

def vector_search(question, milvus_client, embedding_model):
    """向量检索"""
    query_vec = embedding_model.encode([question])
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=query_vec,
        limit=3,
        search_params={"metric_type": "COSINE", "params": {}},
        output_fields=["text"]
    )
    results = [res.entity.get("text") for res in search_res[0]]
    return results

def retrieve_context(question):
    """先编号检索 → fallback 向量检索"""
    article_number = extract_article_number(question)

    if article_number:
        for line in text_lines:
            if article_number in line:
                print(f"找到编号匹配: {article_number}")
                return [line]
        print(f"❓ 未找到编号匹配，使用语义检索")

    results = vector_search(question, milvus_client, embedding_model)
    return results


In [98]:
import os
from openai import OpenAI

api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("没有检测到环境变量 DEEPSEEK_API_KEY，请先设置。")

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1"
)


In [99]:
def ask_deepseek(question, contexts):
    SYSTEM_PROMPT = """
你是一个 AI 法律助手。你能够从提供的上下文段落中找到问题的答案。
如果上下文中没有答案，请直接回答“未找到”。
"""

    USER_PROMPT = f"""
请根据 <context> 中的内容回答 <question> 中的问题。如果找不到答案，请回答“未找到”。
<context>
{"\n".join(contexts)}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

    response = deepseek_client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT},
        ],
    )

    if response and response.choices:
        content = response.choices[0].message.content.strip()
        print("\nDeepSeek 回答：")
        print(content)
    else:
        print("DeepSeek 没有返回结果")


In [100]:
question = "民法典的第二百三十三条内容是什么？"

contexts = retrieve_context(question)

print("\n检索结果:")
for i, ctx in enumerate(contexts, 1):
    print(f"【Top{i}】 {ctx}\n{'-'*50}")

# 调用 deepseek
ask_deepseek(question, contexts)


找到编号匹配: 第二百三十三条

检索结果:
【Top1】 **第二百三十三条** 善意取得。 无处分权人将不动产或者动产转让给受让人，受让人取得该不动产或者动产时是善意，且该不动产或者动产已经登记或者交付的，受让人取得该不动产或者动产的所有权。 受让人依照前款规定取得不动产或者动产的所有权的，原权利人有权向无处分权人请求损害赔偿。 当事人善意取得其他物权的，参照适用前两款规定。
--------------------------------------------------

DeepSeek 回答：
民法典的第二百三十三条内容是关于善意取得的规定，具体如下：

**第二百三十三条** 善意取得。 无处分权人将不动产或者动产转让给受让人，受让人取得该不动产或者动产时是善意，且该不动产或者动产已经登记或者交付的，受让人取得该不动产或者动产的所有权。 受让人依照前款规定取得不动产或者动产的所有权的，原权利人有权向无处分权人请求损害赔偿。 当事人善意取得其他物权的，参照适用前两款规定。


In [101]:
question = "什么是善意取得？"

contexts = retrieve_context(question)

print("\n检索结果:")
for i, ctx in enumerate(contexts, 1):
    print(f"【Top{i}】 {ctx}\n{'-'*50}")

# 调用 deepseek
ask_deepseek(question, contexts)



检索结果:
【Top1】 **第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。 占有人非法占有动产的，善意取得人取得该动产所有权。
--------------------------------------------------
【Top2】 **第二百三十三条** 善意取得。 无处分权人将不动产或者动产转让给受让人，受让人取得该不动产或者动产时是善意，且该不动产或者动产已经登记或者交付的，受让人取得该不动产或者动产的所有权。 受让人依照前款规定取得不动产或者动产的所有权的，原权利人有权向无处分权人请求损害赔偿。 当事人善意取得其他物权的，参照适用前两款规定。
--------------------------------------------------
【Top3】 **第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。 占有人善意占有物的，不承担赔偿责任。 占有人恶意占有物的，应当承担赔偿责任。
--------------------------------------------------

DeepSeek 回答：
根据上下文，善意取得是指无处分权人将不动产或者动产转让给受让人时，如果受让人在取得该不动产或动产时是善意的（即不知道或不应当知道转让人无处分权），并且该不动产或动产已经完成登记或交付手续，那么受让人可以取得该不动产或动产的所有权。原权利人有权向无处分权人请求损害赔偿。此外，善意取得的规定也适用于其他物权的取得。
